# Transforming Code into Beautiful, Idiomatic Python
### with Raymond Hettinger

## 1. Looping Idioms
---

Python's `for` is different from other languages' `for` in that it loops over collections. It may be clearer to regard that keyword as "for each".

```python
# Looping over a list of numbers is straight-forward.
for i in [0, 1, 2, 3, 4, 5]:
    print i**2

# The list is a range of numbers, so this is more clear.
for i in range(6):
    print i**2

# If the list is large, it will consume a lot of memory.
# The best practice in Python 2 is to use the built-in
# xrange, which returns a generator expression that lazily
# computes the following item in the collection.
for i in xrange(6):
    print i**2
```

```python
# Looping over a collection.
colors = ['red', 'green', 'blue', 'yellow']

for color in colors:
    print color
    
# How about looping backwards?
for color in reversed(colors):
    print color
    
# And looping over items and their indices?
for i, color in enumerate(colors):
    print i, '-->', color
```

```python
# Looping over two collections.
names = ['raymond', 'rachel', 'matthew']

for name, color in zip(names, colors):
    print name, '-->', color
    
# But this creates a new collection of tuples in memory,
# which doesn't scale well to larger collections. Rather,
# the izip function is preferred, returning a generator.
import itertools

for name, color in itertools.izip(names, colors):
    print name, '-->', color
```

```python
# Looping in sorted order.
for color in sorted(colors):
    print color

# And reverse sorted order?
for color in sorted(colors, reverse=True):
    print color
```

```python
# Looping with a custom sort order.
# Rather than creating a custom comparison function
# in the likes of qsort, specify the key function to sort.
# In regards to efficiency, this key is called once per element.
print sorted(colors, key=len)
```

```python
# Calling a function until a sentinel value.

# The following loop reads a file in blocks of 32 bytes until
# empty string is reached, appending each block to a list.
import functools

blocks = []
for block in iter(functools.partial(f.read, 32), ''):
    blocks.append(block)

# The first argument of iter is a function of zero arguments.
```

```python
# Distinguishing multiple exit points in loops.
# else clauses on for loops.
def find(seq, target):
    for i, value in enumerate(seq):
        if value == target:
            break
    else:
        return -1
    return 1
```

## 2. Dictionary Skills
---

```python
d = {'matthew': 'blue', 'rachel': 'green', 'raymond': 'red'}

# Looping over keys.
for k in d:
    print k
    
# What about looping over the keys with the
# purpose of mutating the dictionary?
for k in d.keys():
    if k.startswith('r'):
        del d[k]

# This can be expressed as a dictionary comprehension.
d = {k : d[k] for k in d if not k.startswith('r')}
```

```python
# Looping over keys and values.
for k, v in d.iteritems():
    print k, '-->', v
# As you may suspect, the iteritems() method is preferred
# over the items() method because it returns a generator.
```

```python
# Construct a dictionary from pairs.
import itertools

names = ['raymond', 'rachel', 'matthew']
colors = ['red', 'green', 'blue']

d = dict(itertools.izip(names, colors))
```

```python
# Counting with dictionaries.
colors = ['red', 'green', 'red', 'blue', 'green' ,'red']

d = {}
for color in colors:
    if color not in d:
        d[color] = 0
    d[color] += 1
    
# The basic way works and is well-defined, because the key
# we are accessing should exist in the dictionary. A better
# way is to use get, which allows a default value should
# the key not exist.

d= {}
for color in colors:
    d[color] = d.get(color, 0) + 1
    
# An even more modern way is a defaultdict.
import collections

d = collections.defaultdict(int)
for color in colors:
    d[color] += 1
```

```python
# Grouping with Dictionaries
names = ['raymond', 'rachel', 'matthew', 'roger', 'betty', 'melissa', 'judith', 'charlie']

# The following idiom is proven, and is readily
# adapted by modifying the key to suit your purpose.
d = {}
for name in names:
    key = len(name)
    if key not in d:
        d[key] = []
    d[key].append(name)
    
# There is yet a better way, but the method name
# doesn't make the behaviour apparent.
d = {}
for name in names:
    key = len(name)
    d.setdefault(key, []).append(name)

# And as expected, this is the modern way.
import collections

d = collections.defaultdict(list)
for name in names:
    key = len(name)
    d[key].append(name)

```

```python
# The dictionary popitem() method is atomic.
d = {'matthew': 'blue', 'rachel': 'green', 'raymond': 'red'}

while d:
    key, value = d.popitem()
    print key, '-->', value
```

```python
# Linking dictionaries.
import argparse

defaults = {'color': 'red', 'user': 'guest'}
parser = argparse.ArgumentParser()
parser.add_argument('-u', '--user')
parser.add_argument('-c', '--color')
namespace = parser.parse_args([])
command_line_args = {k : v for k, v in vars(namespace).items if v}

d = defaults.copy()
d.update(os.environ)
d.update(command_line_args)

# In Python 3, this has a new idiom.
import collections

d = collections.ChainMap(command_line_args, os.environ, defaults)
```

## 3. Improving Clarity
---

```python
# Clarify function calls with keyword arguments.
twitter_search('@obama', retweets=False, numtweets=20, popular=True)
```

```python
# Clarify multiple return values with named tuples.
TestResults = namedtuple('TestResults', ['failed', 'attempted'])
```

```python
# Unpacking sequences.
p = 'Raymond', 'Hettinger', 0x30, 'python@example.com'

fname, lname, age, email = p
```

```python
# Updating multiple state variables.
def fibonacci(n):
    x, y = 0, 1
    for i in xrange(n):
        print x
        x, y = y, x+y
        
# Update state variables at the same time to avoid
# errors due to out of order updates.
x, y, dx, dy = (x + dx * t,
                y + dy * t,
                influence(m, x, y, dx, dy, partial='x'),
                influence(m, x, y, dx, dy, partial='y'))
```

## 4. Efficiency
---

Don't move data around unnecessarily. It only takes a little care to avoid O(n^2) behaviour instead of linear behaviour.

```python
# Concatenating Strings
names = ['raymond', 'rachel', 'matthew', 'roger', 'betty', 'melissa', 'judith', 'charlie']

# NO
s = names[0]
for name in names[1:]:
    s += ', ' + name
print s

# YES
print ', '.join(names)
```

```python
# Updating Sequences
names = ['raymond', 'rachel', 'matthew', 'roger', 'betty', 'melissa', 'judith', 'charlie']

# NO
del names[0]
names.pop(0)
names.insert(0, 'mark')

# YES
names = deque(['raymond', 'rachel', 'matthew', 'roger', 'betty', 'melissa', 'judith', 'charlie'])

# A double-ended queue allows the following to be performed efficienctly.
del names[0]
names.popleft()
names.appendleft('mark')
```

## 5. Decorators and Context Managers
---

```python
# Using decorators to factor out administrative logic.
import urllib

# Mixes business logic of opening a url and returning a webpage
# with the administrative logic of caching it in a dictionary.
def web_lookup(url, saved={}):
    if url in saved:
        return saved[url]
    page = urllib.urlopen(url).read()
    saved[url] = page
    return page
    
# Simple fix.
def cache(func):
    saved = {}
    @wraps(func)
    def newfunc(*args):
        if args in saved:
            return newfunc(args)
        result = func(*args)
        saved[args] = result
        return result
    return newfunc

@cache
def web_lookup(url):
    return urllib.urlopen(url).read()
```

```python
# Factor out temporary contexts.

# Saving and restoring contexts is a common occurrence.
old_context = getcontext().copy()
getcontext().prec = 50
print Decimal(355) / Decimal(113)
setcontext(old_context)

# Use a context manager to factor out the setup and
# tear-down logic.
with localcontext(Context(prec=50)):
    print Decimal(355) / Decimal(113)
    
# Same for accessing files.
with open('data.txt') as f:
    data = f.read()
```

```python
# How to use locks.
import threading

lock = threading.Lock()

with lock:
    pass
```

```python
# In Python 3.4...
with ignored(OSError):
    os.remove('somefile.tmp')
    
# But in case you want to backport this...
import contextlib

@contextlib.contextmanager
def ignored(*exceptions):
    try:
        yield
    except:
        pass
```

```python
# Redirecting standard output to file.
import contextlib

@contextlib.contextmanager
def redirect_stdout(fileobj):
    oldstdout = sys.stdout
    sys.stdout = fileobj
    try:
        yield fileobj
    finally:
        sys.stdout = oldstdout

with open('help.txt', 'w') as f:
    with redirect_stdout(f):
        help(pow)
```

## 6. Concise, Expressive One-Liners
---

It's a balance between:
1. Not putting too much on one line.
2. Not breaking atoms of thought into subparticles.

Raymond's Rule: One logical line of code equals one English sentence.

```python
# List Comprehensions and Generator Expressions
print sum([i**2 for i in xrange(10)])

# The better way.
print sum(i**2 for i in xrange(10))
```